## 반도체 소자 이상 탐지 AI 경진대회

- 알고리즘 ㅣ 월간데이콘 ㅣ 비전 ㅣ 비지도학습 ㅣ 이상 탐지 ㅣF1 Score

In [ ]:
# Dataset Info.

# train [폴더]
# 학습용 정상 반도체 소자 이미지 샘플 213개
# TRAIN_000.png ~ TRAIN_212.png


# test [폴더]
# 평가용 반도체 소자 이미지 샘플 100개
# TEST_000.png ~ TEST_099.png


# train.csv [파일] 
# ID : 샘플 ID
# img_path : 반도체 소자 이미지 파일 경로
# label : 0 (정상)


# test.csv [파일]
# ID : 샘플 ID
# img_path : 반도체 소자 이미지 파일 경로


# sample_submission.csv [파일] - 제출 양식
# ID : 샘플 ID
# label : 0 (정상), 1 (비정상)


# 데이터는 Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License (CC BY-NC-SA 4.0)에 따라 공개됩니다.

# Paul Bergmann, Kilian Batzner, Michael Fauser, David Sattlegger, Carsten Steger: The MVTec Anomaly Detection Dataset: A Comprehensive Real-World Dataset for Unsupervised Anomaly Detection; in: International Journal of Computer Vision 129(4):1038-1059, 2021, DOI: 10.1007/s11263-020-01400-4.

# Paul Bergmann, Michael Fauser, David Sattlegger, Carsten Steger: MVTec AD — A Comprehensive Real-World Dataset for Unsupervised Anomaly Detection; in: IEEE/CVF Conference on Computer Vision and Pattern Recognition (CVPR), 9584-9592, 2019, DOI: 10.1109/CVPR.2019.00982.



## Import Functions

In [1]:
import pandas as pd
import numpy as np
import torch
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from sklearn.ensemble import IsolationForest
from tqdm import tqdm

# GPU 사용 설정

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')